# Getting Started with remote-store

This notebook walks through the basics: setting up a configuration, creating a registry, getting a store, and performing your first read/write operations.

## Setup

We use a temporary directory so this notebook is self-cleaning.

In [ ]:
import tempfile

from remote_store import BackendConfig, Registry, RegistryConfig, StoreProfile

# Create a temp directory that we'll clean up at the end
tmpdir = tempfile.mkdtemp()
print(f"Working directory: {tmpdir}")

## Create a Configuration

A `RegistryConfig` describes your backends and stores. Here we define a single local backend and one store scoped to a `data/` prefix.

In [ ]:
config = RegistryConfig(
    backends={"local": BackendConfig(type="local", options={"root": tmpdir})},
    stores={"data": StoreProfile(backend="local", root_path="data")},
)
config

## Open a Registry and Get a Store

The `Registry` validates your config and provides named stores. Stores are the primary API for file operations.

In [ ]:
registry = Registry(config)
store = registry.get_store("data")
print(f"Store type: {type(store).__name__}")

## Write and Read a File

In [ ]:
store.write("hello.txt", b"Hello from the notebook!")
print(f"File exists: {store.exists('hello.txt')}")

content = store.read_bytes("hello.txt")
print(f"Content: {content.decode()}")

## Inspect File Metadata

In [ ]:
info = store.get_file_info("hello.txt")
print(f"Name: {info.name}")
print(f"Size: {info.size} bytes")
print(f"Modified: {info.modified_at}")
print(f"Path: {info.path}")

## Write More Files and List Them

In [ ]:
store.write("notes/day1.md", b"# Day 1\nGot started!")
store.write("notes/day2.md", b"# Day 2\nMade progress!")

print("Files in notes/:")
for f in store.list_files("notes"):
    print(f"  {f.name} ({f.size} bytes)")

print("\nAll notes (recursive):")
for f in store.list_files("notes", recursive=True):
    print(f"  {f.path} ({f.size} bytes)")

## Cleanup

In [ ]:
import shutil

registry.close()
shutil.rmtree(tmpdir)
print("Cleaned up.")